In [1]:
import cv2
import numpy as np
import cvzone
from supervision.video.dataclasses import VideoInfo
from supervision.video.sink import VideoSink
import torch
import math
import pandas as pd
from xgboost import XGBRegressor
from shapely.geometry import LineString

In [2]:
def draw_bboxes(image, bbox,classes=None,colors=None):
    img = image.copy()
    for idx,box in enumerate(bbox):
        # get the (x, y) coordinate for the center of the bounding box,
        # and the width and height of the box
        x, y, w, h = (box[0], box[1],box[2], box[3])

        # convert the x, y, w, h values to integer
        x, y, w, h = int(x), int(y), int(w), int(h)

        # calculate the starting and ending (x, y) coordinates for the bounding box
        start_x, end_x = x - int(w / 2), x + int(w / 2)
        start_y, end_y = y - int(h / 2), y + int(h / 2)

        # draw the bounding box on the image
        color = (0,255,0)
        if classes is not None:
            color = colors[int(classes[idx])]
        
        cv2.rectangle(img, (start_x, start_y), (end_x, end_y), color, 2)
        
    return img


def draw_pockets(pockets,image):
    img = image.copy()
    LT_pocket,LB_pocket,RT_pocket,RB_pocket,TC_pocket,BC_pocket = pockets
    
    cv2.line(img,LT_pocket[0],LT_pocket[1],(0,255,0),5)
    cv2.line(img,LB_pocket[0],LB_pocket[1],(0,255,0),5)
    cv2.line(img,RT_pocket[0],RT_pocket[1],(0,255,0),5)
    cv2.line(img,RB_pocket[0],RB_pocket[1],(0,255,0),5)
    cv2.line(img,TC_pocket[0],TC_pocket[1],(0,255,0),5)
    cv2.line(img,BC_pocket[0],BC_pocket[1],(0,255,0),5)
    
    return img
    
    

In [3]:
def euclidean_distance(point1, point2):
    x1, y1 = point1
    x2, y2 = point2
    distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return distance


def detect_movement(prev_distance,xy,threshold):
    if len(xy) > 1:
        distance = euclidean_distance(tuple(xy[0]),tuple(xy[1]))
    else:
        distance = None
        
    distance_away = None
    if distance is not None and prev_distance is not None:
        distance_away = int(abs(prev_distance - distance))
        
    if distance_away is not None:
        if distance_away < threshold:
            return False,distance
    
    return True,distance


def reset(classes,preds,prev_pos,threshold=100):
    index = torch.where(classes == 1)[0]
    index = int(index[0]) if len(index) >= 1 else None
    
    if index is not None:
        x,y,w,h = preds[index]
        if prev_pos is not None:
            distance = euclidean_distance((x,y),prev_pos)
            if distance > threshold:
                return True,(x,y)
            else:
                return False,(x,y)
        else:
            return True, (x,y)
        
    return True,None

In [4]:
def get_line_boarder(point1,point2,board_ltrb):
    # Calculate the slope of the line
    l,t,r,b = board_ltrb
    slope = (point2[1] - point1[1]) / (point2[0] - point1[0])
    # Calculate the y-intercept of the line (i.e., the value of y when x=0)
    intercept = point1[1] - slope * point1[0]
    
    if point2[1] > point1[1]:
        y = b
        x = int((y-intercept)//slope)
        
        if l < x < r:
            pred_x = x
            pred_y = y
            
        elif x < l:
            pred_x = l
            pred_y = slope*pred_x + intercept
        
        else:
            pred_x = r
            pred_y = slope*pred_x + intercept
    
    else:
        y = t
        x = int((y-intercept)//slope)
        
        if l < x < r:
            pred_x = x
            pred_y = y
            
        elif x < l:
            pred_x = l
            pred_y = slope*pred_x + intercept
        
        else:
            pred_x = r
            pred_y = slope*pred_x + intercept
        
    return (int(pred_x),int(pred_y))


def get_perpendicular_dist(line,point):
    # Define the line segment
    start_point,end_point = line

    # Find the equation of the line segment
    try:
        slope = (end_point[1] - start_point[1]) / (end_point[0] - start_point[0])
    except:
        return 0

    intercept = start_point[1] - slope * start_point[0]
    A = slope
    B = -1
    C = intercept

    x, y = point

    # Find the distance between the point and the line segment
    distance = abs(A * x + B * y + C) / math.sqrt(A ** 2 + B ** 2)
    
    return distance

In [12]:
def extend_point(point1,point2,DPF,model,board):
    # calculate the slope and intercept of the line
    x1, y1 = point1
    x2, y2 = point2
    m = float(((y2 - y1) / (x2 - x1)))
    b = float(y1 - m * x1)

    # calculate the direction of the line
    theta = math.atan2(y2 - y1, x2 - x1) #+ 0.1
    new_data = pd.DataFrame({'DPF': [DPF], 'init_theta': [theta]})
    predicted = model.predict(new_data)[0]
    
    # calculate the length of the line
    length = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

    # extend the line by a factor of 2
    extended_length = length * 20
    
    x2_extended = int(x2 + extended_length * math.cos(theta+predicted))
    y2_extended = int(y2 + extended_length * math.sin(theta+predicted))
    
    d = (int(x1) + (int(y1) - b)*m)/(1 + m**2)
    
    l,t,w,h = board
    l,t,r,b = l,t,l+w,t+h
    x2,y2 = get_line_boarder(point1,(x2_extended,y2_extended),(l,t,r,b))
    x3,y3 = get_line_boarder(point1,(x2_extended,y2_extended),(l+55,t+50,l+w-55,t+h-55))
    
    return ((int(x1),int(y1)),(int(x2),int(y2))),((int(x1),int(y1)),(int(x3),int(y3))),theta


def generate_points(point1,point2,step):
    # Calculate the slope of the line
    slope = (point2[1] - point1[1]) / (point2[0] - point1[0])
    # Calculate the y-intercept of the line (i.e., the value of y when x=0)
    intercept = point1[1] - slope * point1[0]
    
    if point2[1] > point1[1]:
        ys = [y for y in range(point2[1],point2[1]+(20*step),step)]
        xs = [int((y-intercept)//slope) for y in ys]
    
    else:
        ys = reversed(list(range(max(0, point2[1]-(20*step)),point2[1],step)))
        ys = [y for y in ys]
        xs = [int((y-intercept)//slope) for y in ys]
        
    points = [(x,y) for x,y in zip(xs,ys)]
    
    return points


def get_pockets(board):
    l,t,w,h = board
    
    LT_pocket = ((l+30,t+75) ,(l+50+30,t+25))
    LB_pocket = ((l+30,t+h-60),(l+50+30,t+h-25))
    RT_pocket = ((l+w-20,t+50) ,(l+w-50-25,t+10))
    RB_pocket = ((l+w-75,t+h-20),(l+w-30,t+h-75))
    TC_pocket = (((l+l+w)//2 - 45 ,t+25),((l+l+w)//2 + 45 ,t+25))
    BC_pocket = (((l+l+w)//2 - 45 ,t+h-25),((l+l+w)//2 + 45 ,t+h-25))
    
    return (LT_pocket,LB_pocket,RT_pocket,RB_pocket,TC_pocket,BC_pocket)


def predict_shot(pockets,BT_line):
    shot = False
    big_line = LineString([BT_line[0], BT_line[1]])
    for pocket in pockets:
        small_line = LineString([pocket[0], pocket[1]])
        if big_line.intersects(small_line):
            shot = True
            break
            
    return shot


def gradient(pt1,pt2):
    if pt1[0] == pt2[0] or pt1[1] == pt2[1]:
        return 0
    return (pt2[1]-pt1[1])/(pt2[0]-pt1[0])

def select_line(board_ltrb,point):
    points = {
        "l": board_ltrb[0],
        "t": board_ltrb[1],
        "r": board_ltrb[2],
        "b": board_ltrb[3]
    }
    line = None
    line_pos = None
    for key,n in points.items():
        if n in point:
            if key == "l" or key == "r":
                line = ((n,points["t"]) ,(n,points["b"]))
                line_pos = key
            else:
                line = ((points["l"],n) ,(points["r"],n))
                line_pos = key
            break
            
    return line,line_pos


def get_reflection_point(draw_line,board_ltrb,DPF,model,length = 10):
    l,t,r,b = board_ltrb
    selected_line,line_pos = select_line((l,t,r,b),draw_line[1])
    print(selected_line)

    m1 = gradient(selected_line[0],selected_line[1])
    m2 = gradient(draw_line[0],draw_line[1])
    
    if m1 == None or m2 == None:
        return None
    
    angR = math.atan((m2-m1)/(1+(m2*m1))) # find taninverse, it will return it as radians
    angD = round(math.degrees(angR))
    
    init_angle = 180 - angD
    start_point = draw_line[1]
    
    init_endpoint = (int(start_point[0] + length * np.cos(init_angle * np.pi / 180)),
                 int(start_point[1] + length * np.sin(init_angle * np.pi / 180)))
    
    if not ((r+30 >= init_endpoint[0] >= l-30) and  (b+40 >= init_endpoint[1] >= t-30)):
        dx,dy = start_point[0] - init_endpoint[0] ,start_point[1] - init_endpoint[1]
        init_endpoint = (start_point[0] + dx ,start_point[1] + dy)
    
    end_point1 = get_line_boarder(start_point,init_endpoint,(l,t,r,b))
    length = euclidean_distance(start_point,end_point1)

    new_data = pd.DataFrame({'DPF': [DPF], 'init_angle': [init_angle], 'line_length':[length]})
    predicted = model.predict(new_data)[0]

    new_angle = init_angle + predicted
    end_point = (int(start_point[0] + length * np.cos(new_angle * np.pi / 180)),
                 int(start_point[1] + length * np.sin(new_angle * np.pi / 180)))
    if not ((r+30 >= end_point[0] >= l-30) and  (b+40 >= end_point[1] >= t-30)):
        dx,dy = start_point[0] - end_point[0] ,start_point[1] - end_point[1]
        end_point = (start_point[0] + dx ,start_point[1] + dy)
        
    return end_point 

In [13]:
from supervision.video.dataclasses import VideoInfo
from supervision.video.sink import VideoSink

In [14]:

SOURCE_VIDEO_PATH = "Shot-Predictor-Video.mp4"
TARGET_VIDEO_PATH = "./poolball.mp4"

video_info = VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
video_info

VideoInfo(width=1920, height=1080, fps=30, total_frames=1222)

In [37]:
# Post Processing Step to calculate the exact value,


DPF = [30.7,78.7,50.5,69.3,52.9,45.6,51.4,76.577,41.369,58.1]
init_theta = [-2.27,1.28,-2.46,1.07,1.31,-1.938,1.689,0.849,2.22,1.45]
theta_increment = [0,0,0,0.1,0,0.2,0,0,0.07,0]

data = pd.DataFrame({'DPF': DPF, 'init_theta': init_theta, 'increment': theta_increment})

X = data[['DPF', 'init_theta']]
y = data['increment']

thetha_model = XGBRegressor(n_estimators=100, max_depth=3, learning_rate=0.1, subsample=0.8)
thetha_model.fit(X, y)


DPF = [78.7,69.3,52.9,45.6,76.577,58.1]
init_angle = [106,113,105,102,131,95]
line_length = [694,728,156,321,903,693]
angle_increment = [-4,2,0,3,6,13]

data = pd.DataFrame({'DPF': DPF, 'init_angle': init_angle,'line_length':line_length, 'angle_increment': angle_increment})

X = data[['DPF','init_angle','line_length']]
y = data['angle_increment']

angle_model = XGBRegressor(n_estimators=100, max_depth=3, learning_rate=0.1, subsample=0.8)
angle_model.fit(X, y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=4,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=0.8,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [38]:
from ultralytics import YOLO
model = YOLO("train/weights/best.pt",task="detect")

In [39]:
class Args:
    def __init__(self):
        self.prev_distance = None #N
        self.prev_W_pos = None #N
        self.collided = False #N
        self.predicted = False #N
        self.distance_per_frame = None #N
        self.is_reset = True #N
        self.white_ball = None #N
        self.target_ball = None #N
        self.selected_pos = None #N
        self.all_pockets = None #N
        self.board = None #N
        self.is_shot = None #N
        self.tragectory_lines = [] #N
        self.all_predictions = [] #N

In [61]:
hsvVals = {'hmin': 55, 'smin': 72, 'vmin': 49, 'hmax': 98, 'smax': 255, 'vmax': 143}
color_finder = cvzone.ColorFinder(trackBar=False)

video = cv2.VideoCapture(SOURCE_VIDEO_PATH)

colors = {
    0: (0,255,0),
    1: (0,0,255)
}

args = Args()

with VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
    while True:
        _,frame = video.read()
        if frame is None:
            break
        
        if args.board is None:
            ROI_area,mask = color_finder.update(frame,hsvVals)
            contour_img,contours_data = cvzone.findContours(frame,mask,minArea=10000)
            if len(contours_data) > 0:
                args.board = contours_data[0]["bbox"]
                args.all_pockets = get_pockets(args.board)
                
            
        cv2.imwrite("frame.jpg",frame)
        results = model.predict(source="frame.jpg", conf=0.5)
        preds = results[0].boxes.xywh
        classes = results[0].boxes.cls

        if len(preds) > 0:
            max_radius = torch.max(preds[:,2:]).cpu().numpy().item()//2
            xy = preds[:,:2].cpu().numpy()

            is_moved,distance = detect_movement(args.prev_distance,xy,threshold=10)
            args.is_reset,W_pos = reset(classes,preds,args.prev_W_pos,threshold=100)

            if (distance is not None and distance < (max_radius*2)+15 and not args.collided):
                args.collided = True
                index = torch.where(classes == 0)[0]
                index = int(index[0])
                x,y,w,h = preds[index]
                args.distance_per_frame = euclidean_distance(args.prev_W_pos,W_pos)

            if args.is_reset:
                index1 = torch.where(classes == 1)[0]
                index2 = torch.where(classes == 0)[0]

                if len(index1) > 0 and len(index2) > 0:
                    args = Args()
                    index1 = int(index1[0])
                    args.white_ball = preds[index1]
                    index2 = int(index2[0])
                    args.target_ball = preds[index2]
                else:
                    continue
                

            elif args.collided and not args.predicted:
                end_point = (int(W_pos[0]) ,int(W_pos[1]))
                x,y,w,h = args.white_ball
                args.all_predictions = generate_points((int(x),int(y)),end_point,step=2)
                x,y,w,h = args.target_ball
                for idx,pos in enumerate(args.all_predictions):
                    ball_distance = euclidean_distance((x,y),(pos))
                    if ball_distance <= w+15:
                        args.selected_pos = args.all_predictions[idx]
                        line1,draw_line,theta = extend_point(args.selected_pos,(x,y),args.distance_per_frame,thetha_model,args.board)
                        args.tragectory_lines.append(draw_line)
                        args.is_shot = predict_shot(args.all_pockets,line1)
                        
                        if not args.is_shot:
                            l,t,w,h = args.board
                            o_l,o_t,o_r,o_b = l,t,l+w,t+h
                            l,t,r,b = l+55,t+50,l+w-55,t+h-55
                            selected_line,pos_n = select_line((l,t,r,b),draw_line[1])
                            dist = int(get_perpendicular_dist(selected_line,(int(x),int(y))))
                            if dist < 35:
                                l,t,r,b = o_l+25,o_t+25,o_r-25,o_b-35
                                draw_line = (draw_line[0],get_line_boarder(draw_line[0],draw_line[1],(l,t,r,b)))

                            end_point= get_reflection_point(draw_line,(l,t,r,b),args.distance_per_frame,angle_model,length = 100)
                            if end_point is not None:
                                end_point = get_line_boarder(draw_line[1],end_point,(o_l,o_t,o_r,o_b))
                                
                                end_point_draw = get_line_boarder(draw_line[1],end_point,(l,t,r,b))
                                line2 = ((draw_line[1],end_point_draw))
                                args.is_shot = predict_shot(args.all_pockets,((draw_line[1],end_point)))
                                args.tragectory_lines.append(line2)                        
                        break
                args.predicted = True



            args.prev_distance = distance
            args.prev_W_pos = W_pos        

                                

            if args.selected_pos is not None:
                prev_point = None
                for line in args.tragectory_lines:
                    if prev_point is None:
                        cv2.line(frame,line[0],line[1],colors[not args.is_shot],5)
                        prev_point = line[1]
                    else:
                        cv2.line(frame,prev_point,line[1],colors[not args.is_shot],5)
                    

                    cv2.circle(frame,line[1],int(args.white_ball[2])//2,colors[not args.is_shot],-1)
                    cv2.circle(frame,line[1],int(args.white_ball[2])//2,(0,0,0),3)
                    
                x,y,w,h = args.target_ball
                cv2.circle(frame,(int(args.selected_pos[0]),int(args.selected_pos[1])),int(args.white_ball[2])//2,(255,255,255),-1)
                cv2.circle(frame,(int(x),int(y)),int(args.white_ball[2])//2,colors[not args.is_shot],-1)
                cv2.circle(frame,(int(args.selected_pos[0]),int(args.selected_pos[1])),int(args.white_ball[2])//2,(0,0,0),3)
                cv2.circle(frame,(int(x),int(y)),int(args.white_ball[2])//2,(0,0,0),3)
                cv2.rectangle(frame,(300,15),(850,75),(0,0,0),-1)
                cv2.putText(frame,f"Prediction: {'IN' if args.is_shot else 'OUT'}",(335,70),4,1.5,colors[not args.is_shot],3)
                
            # cv2.putText(frame, f"Moved: {is_moved}", (50, 50), 2, 2, (255,0,0), 3)
            # cv2.putText(frame, f"Reset: {args.is_reset}", (50, 150), 2, 2, (255,255,0), 3)
            # cv2.putText(frame, f"Collide: {args.collided}", (50, 250), 2, 2, (255,255,0), 3)
            # cv2.putText(frame, f"is_shot: {args.is_shot}", (50, 350), 2, 2, (255,255,0), 3)


        
        # image = draw_bboxes(frame,preds,classes,colors)
        image = frame.copy()
        # if args.board is not None:
        #     image = draw_pockets(args.all_pockets,image)
        #     l,t,w,h = args.board
        #     l,t,r,b = l+55,t+50,l+w-55,t+h-55
        #     cv2.rectangle(image, (l, t), (r, b), (0,0,255), 2)

        
        sink.write_frame(image)
        resized_image = cv2.resize(image,(640,480),interpolation=cv2.INTER_AREA)
        cv2.imshow("Pool",resized_image)
        key = cv2.waitKey(1)

        if key == 115:
            break
            
video.release()
cv2.destroyAllWindows()


image 1/1 /home/zbook/Jupyter_Notebooks/Competitions/Pool Tragectory Predictor/frame.jpg: 384x640 1 color-ball, 1 white-ball, 3764.3ms
Speed: 5.3ms preprocess, 3764.3ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)
QObject::moveToThread: Current thread (0x55e533088d40) is not the object's thread (0x55e52c9138c0).
Cannot move to target thread (0x55e533088d40)

QObject::moveToThread: Current thread (0x55e533088d40) is not the object's thread (0x55e52c9138c0).
Cannot move to target thread (0x55e533088d40)

QObject::moveToThread: Current thread (0x55e533088d40) is not the object's thread (0x55e52c9138c0).
Cannot move to target thread (0x55e533088d40)

QObject::moveToThread: Current thread (0x55e533088d40) is not the object's thread (0x55e52c9138c0).
Cannot move to target thread (0x55e533088d40)

QObject::moveToThread: Current thread (0x55e533088d40) is not the object's thread (0x55e52c9138c0).
Cannot move to target thread (0x55e533088d40)

QObject::moveToThread: Current


image 1/1 /home/zbook/Jupyter_Notebooks/Competitions/Pool Tragectory Predictor/frame.jpg: 384x640 1 color-ball, 1 white-ball, 1129.8ms
Speed: 11.1ms preprocess, 1129.8ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/zbook/Jupyter_Notebooks/Competitions/Pool Tragectory Predictor/frame.jpg: 384x640 1 color-ball, 1 white-ball, 2375.8ms
Speed: 1.0ms preprocess, 2375.8ms inference, 11.3ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/zbook/Jupyter_Notebooks/Competitions/Pool Tragectory Predictor/frame.jpg: 384x640 1 color-ball, 1 white-ball, 4798.2ms
Speed: 58.9ms preprocess, 4798.2ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/zbook/Jupyter_Notebooks/Competitions/Pool Tragectory Predictor/frame.jpg: 384x640 1 color-ball, 1 white-ball, 1493.1ms
Speed: 23.1ms preprocess, 1493.1ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/zbook/Jupyter_Notebooks/Competitions/Po

Speed: 19.2ms preprocess, 2204.1ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/zbook/Jupyter_Notebooks/Competitions/Pool Tragectory Predictor/frame.jpg: 384x640 1 color-ball, 1 white-ball, 608.4ms
Speed: 19.6ms preprocess, 608.4ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/zbook/Jupyter_Notebooks/Competitions/Pool Tragectory Predictor/frame.jpg: 384x640 1 color-ball, 1 white-ball, 763.6ms
Speed: 16.8ms preprocess, 763.6ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/zbook/Jupyter_Notebooks/Competitions/Pool Tragectory Predictor/frame.jpg: 384x640 1 color-ball, 1 white-ball, 1486.9ms
Speed: 1.0ms preprocess, 1486.9ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/zbook/Jupyter_Notebooks/Competitions/Pool Tragectory Predictor/frame.jpg: 384x640 1 color-ball, 1 white-ball, 1259.7ms
Speed: 29.7ms preprocess, 1259.7ms inference, 1.3ms postproce

Speed: 2.9ms preprocess, 1111.2ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/zbook/Jupyter_Notebooks/Competitions/Pool Tragectory Predictor/frame.jpg: 384x640 1 white-ball, 1127.1ms
Speed: 1.0ms preprocess, 1127.1ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/zbook/Jupyter_Notebooks/Competitions/Pool Tragectory Predictor/frame.jpg: 384x640 1 white-ball, 1116.8ms
Speed: 1.7ms preprocess, 1116.8ms inference, 4.8ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/zbook/Jupyter_Notebooks/Competitions/Pool Tragectory Predictor/frame.jpg: 384x640 1 white-ball, 1038.5ms
Speed: 3.5ms preprocess, 1038.5ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/zbook/Jupyter_Notebooks/Competitions/Pool Tragectory Predictor/frame.jpg: 384x640 1 white-ball, 1224.1ms
Speed: 1.6ms preprocess, 1224.1ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/

Speed: 1.0ms preprocess, 580.8ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/zbook/Jupyter_Notebooks/Competitions/Pool Tragectory Predictor/frame.jpg: 384x640 1 color-ball, 1 white-ball, 827.2ms
Speed: 1.0ms preprocess, 827.2ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/zbook/Jupyter_Notebooks/Competitions/Pool Tragectory Predictor/frame.jpg: 384x640 1 color-ball, 1 white-ball, 527.2ms
Speed: 1.2ms preprocess, 527.2ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/zbook/Jupyter_Notebooks/Competitions/Pool Tragectory Predictor/frame.jpg: 384x640 1 color-ball, 1 white-ball, 951.0ms
Speed: 1.0ms preprocess, 951.0ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/zbook/Jupyter_Notebooks/Competitions/Pool Tragectory Predictor/frame.jpg: 384x640 1 color-ball, 1 white-ball, 988.8ms
Speed: 3.0ms preprocess, 988.8ms inference, 0.7ms postprocess per im

...


image 1/1 /home/zbook/Jupyter_Notebooks/Competitions/Pool Tragectory Predictor/frame.jpg: 384x640 1 color-ball, 1 white-ball, 479.6ms
Speed: 1.1ms preprocess, 479.6ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/zbook/Jupyter_Notebooks/Competitions/Pool Tragectory Predictor/frame.jpg: 384x640 1 color-ball, 1 white-ball, 493.3ms
Speed: 1.0ms preprocess, 493.3ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/zbook/Jupyter_Notebooks/Competitions/Pool Tragectory Predictor/frame.jpg: 384x640 1 color-ball, 1 white-ball, 481.3ms
Speed: 1.0ms preprocess, 481.3ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/zbook/Jupyter_Notebooks/Competitions/Pool Tragectory Predictor/frame.jpg: 384x640 1 color-ball, 1 white-ball, 498.9ms
Speed: 1.0ms preprocess, 498.9ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/zbook/Jupyter_Notebooks/Competitions/Pool Tragector

Speed: 1.0ms preprocess, 478.5ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/zbook/Jupyter_Notebooks/Competitions/Pool Tragectory Predictor/frame.jpg: 384x640 1 color-ball, 1 white-ball, 479.8ms
Speed: 1.2ms preprocess, 479.8ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/zbook/Jupyter_Notebooks/Competitions/Pool Tragectory Predictor/frame.jpg: 384x640 1 color-ball, 1 white-ball, 488.4ms
Speed: 1.0ms preprocess, 488.4ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/zbook/Jupyter_Notebooks/Competitions/Pool Tragectory Predictor/frame.jpg: 384x640 1 color-ball, 1 white-ball, 503.2ms
Speed: 1.0ms preprocess, 503.2ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)



((257, 781), (1671, 781))


image 1/1 /home/zbook/Jupyter_Notebooks/Competitions/Pool Tragectory Predictor/frame.jpg: 384x640 1 color-ball, 1 white-ball, 487.1ms
Speed: 1.0ms preprocess, 487.1ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/zbook/Jupyter_Notebooks/Competitions/Pool Tragectory Predictor/frame.jpg: 384x640 1 color-ball, 1 white-ball, 496.3ms
Speed: 1.0ms preprocess, 496.3ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/zbook/Jupyter_Notebooks/Competitions/Pool Tragectory Predictor/frame.jpg: 384x640 1 color-ball, 1 white-ball, 495.3ms
Speed: 1.0ms preprocess, 495.3ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/zbook/Jupyter_Notebooks/Competitions/Pool Tragectory Predictor/frame.jpg: 384x640 2 white-balls, 491.9ms
Speed: 1.1ms preprocess, 491.9ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/zbook/Jupyter_Notebooks/Competitions/Pool Tragectory Predictor/fr


image 1/1 /home/zbook/Jupyter_Notebooks/Competitions/Pool Tragectory Predictor/frame.jpg: 384x640 1 white-ball, 1959.7ms
Speed: 5.6ms preprocess, 1959.7ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/zbook/Jupyter_Notebooks/Competitions/Pool Tragectory Predictor/frame.jpg: 384x640 1 white-ball, 1000.7ms
Speed: 33.6ms preprocess, 1000.7ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/zbook/Jupyter_Notebooks/Competitions/Pool Tragectory Predictor/frame.jpg: 384x640 1 white-ball, 479.7ms
Speed: 2.1ms preprocess, 479.7ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/zbook/Jupyter_Notebooks/Competitions/Pool Tragectory Predictor/frame.jpg: 384x640 1 white-ball, 646.1ms
Speed: 1.0ms preprocess, 646.1ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/zbook/Jupyter_Notebooks/Competitions/Pool Tragectory Predictor/frame.jpg: 384x640 1 white-ball, 711.3m